In [ ]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
import matplotlib.pyplot as plt
import gc; gc.enable() 
print(os.listdir("../input/airbus-ship-detection"))

In [ ]:
masks = pd.read_csv(os.path.join('../input/airbus-ship-detection', 'train_ship_segmentations_v2.csv'))
not_empty = pd.notna(masks.EncodedPixels)
print(not_empty.sum(), 'masks in', masks[not_empty].ImageId.nunique(), 'images')#非空图片中的mask数量
print((~not_empty).sum(), 'empty images in', masks.ImageId.nunique(), 'total images')#所有图片中非空图片
masks.head()

In [ ]:
masks['ships'] = masks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
masks.head()

In [ ]:
unique_img_ids = masks.groupby('ImageId').agg({'ships': 'sum'}).reset_index()
unique_img_ids.head()

In [ ]:
unique_img_ids['has_ship'] = unique_img_ids['ships'].map(lambda x: 1.0 if x>0 else 0.0)

unique_img_ids.head()

In [ ]:
ship_dir = '../input/airbus-ship-detection'
train_image_dir = os.path.join(ship_dir, 'train_v2')
test_image_dir = os.path.join(ship_dir, 'test_v2')
unique_img_ids['has_ship_vec'] = unique_img_ids['has_ship'].map(lambda x: [x])
unique_img_ids['file_size_kb'] = unique_img_ids['ImageId'].map(lambda c_img_id: 
                                                               os.stat(os.path.join(train_image_dir, 
                                                                                    c_img_id)).st_size/1024)
unique_img_ids.head()

In [ ]:
unique_img_ids = unique_img_ids[unique_img_ids['file_size_kb'] > 50] # keep only +50kb files
plt.hist(x = unique_img_ids['file_size_kb'], # 指定绘图数据
           bins = 6, # 指定直方图中条块的个数
           color = 'steelblue', # 指定直方图的填充色
           edgecolor = 'black' # 指定直方图的边框色
          )
plt.xticks([50,100,150,200,250,300,350,400,450,500])
plt.ylabel("number")
plt.xlabel('file_size_kb')
#unique_img_ids['file_size_kb'].hist()#绘制直方图
masks.drop(['ships'], axis=1, inplace=True)
unique_img_ids.sample(7)
plt.title("Number of images of each size")

In [ ]:
train_0 = unique_img_ids[unique_img_ids['ships']==1].sample(1800)
train_1 = unique_img_ids[unique_img_ids['ships']==2].sample(1800)
train_2 = unique_img_ids[unique_img_ids['ships']==3].sample(1800)
train_3 = unique_img_ids[unique_img_ids['ships']!=3]
train_3 = train_3[unique_img_ids['ships']!=2]
train_3 = train_3[unique_img_ids['ships']!=1]

In [ ]:
unique_img_ids=pd.concat([train_0,train_1,train_2,train_3])

In [ ]:
SAMPLES_PER_GROUP = 10000#1500
balanced_train_df = unique_img_ids.groupby('ships').apply(lambda x: x.sample(SAMPLES_PER_GROUP) if len(x) > SAMPLES_PER_GROUP else x)
#图片有相同船舶数量，但超出2000的不要
rect=plt.hist(x = balanced_train_df['ships'], # 指定绘图数据
           bins = 16, # 指定直方图中条块的个数
           color = 'steelblue', # 指定直方图的填充色
           edgecolor = 'black' # 指定直方图的边框色
          )
plt.yticks(range(0,1800,300))#1800
plt.xticks(range(0,15))
plt.ylabel("Number of images")
plt.xlabel('Number of ships')
plt.title("Number of images containing different number of vessels")
#balanced_train_df['ships'].hist(bins=balanced_train_df['ships'].max()+1)
print(balanced_train_df.shape[0], 'images',balanced_train_df.shape)#取出1万张图片


In [ ]:
from PIL import Image
x = np.empty(shape=(20188, 256,256,3),dtype=np.uint8)#10680 256
y = np.empty(shape=20188,dtype=np.uint8)
for index, image in enumerate(balanced_train_df['ImageId']):
    image_array= Image.open('../input/airbus-ship-detection/train_v2/' + image).resize((256,256)).convert('RGB') #256
    x[index] = image_array
    y[index]=balanced_train_df[balanced_train_df['ImageId']==image]['has_ship'].iloc[0]

print(x.shape)
print(y.shape)

In [ ]:
###### Set target to one hot target for classification problem
#为分类问题将目标设置为一个热目标
from sklearn.preprocessing import OneHotEncoder
y_targets =y.reshape(len(y),-1)
enc = OneHotEncoder()
enc.fit(y_targets)
y = enc.transform(y_targets).toarray()
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val  = train_test_split(x,y,test_size = 0.2,random_state=1,stratify=y)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
import warnings
import numpy as np

from keras.preprocessing import image
from keras.models import Model
from keras import layers
from keras.layers import Activation, AveragePooling2D, BatchNormalization, Concatenate
from keras.layers import Conv2D, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, Lambda, MaxPooling2D
from keras.layers import SeparableConv2D, DepthwiseConv2D
from keras.layers import Add, Multiply, Reshape
from keras.applications.imagenet_utils import decode_predictions
from keras.utils.data_utils import get_file
from keras import backend as K

from keras.utils.generic_utils import get_custom_objects


def relu6(x):
    # relu函数
    return K.relu(x, max_value=6.0)


get_custom_objects().update({'relu6': Activation(relu6)})


def hard_swish(x):
    # 利用relu函数乘上x模拟sigmoid
    return x * K.relu(x + 3.0, max_value=6.0) / 6.0


get_custom_objects().update({'hard_swish': Activation(hard_swish)})


def return_activation(x, nl):
    # 用于判断使用哪个激活函数
    if nl == 'HS':
        x = Activation(hard_swish)(x)
    if nl == 'RE':
        x = Activation(relu6)(x)
    return x


def channel_split(x, name=''):
    in_channels = x.shape.as_list()[-1]
    ip = in_channels // 2
    c_hat = Lambda(lambda z: z[:, :, :, 0:ip])(x)
    c = Lambda(lambda z: z[:, :, :, ip:])(x)

    return c_hat, c


def channel_shuffle(x):
    height, width, channels = x.shape.as_list()[1:]
    channels_per_split = channels // 2

    x = K.reshape(x, [-1, height, width, 2, channels_per_split])
    x = K.permute_dimensions(x, (0, 1, 2, 4, 3))
    x = K.reshape(x, [-1, height, width, channels])

    return x


def squeeze(inputs):
    # 注意力机制单元
    input_channels = int(inputs.shape[-1])

    x = GlobalAveragePooling2D()(inputs)
    x = Dense(int(input_channels / 4))(x)
    x = Activation(relu6)(x)
    x = Dense(input_channels)(x)
    x = Activation(hard_swish)(x)
    x = Reshape((1, 1, input_channels))(x)
    x = Multiply()([inputs, x])

    return x


def _shuffle_unit(inputs, out_channels, sq, nl, strides=2, stage=1, block=1):
    bn_axis = -1  # 通道在后还是在前
    prefix = 'stage%d/block%d' % (stage, block)

    branch_channels = out_channels // 2

    if strides == 2:
        x_1 = DepthwiseConv2D(kernel_size=3, strides=2, padding='same',
                              use_bias=False, name='%s/3x3dwconv_1' % prefix)(inputs)
        x_1 = BatchNormalization(axis=bn_axis, name='%s/bn_3x3dwconv_1' % prefix)(x_1)
        x_1 = Conv2D(filters=branch_channels, kernel_size=1, strides=1, padding='same',
                     use_bias=False, name='%s/1x1conv_1' % prefix)(x_1)
        x_1 = BatchNormalization(axis=bn_axis, name='%s/bn_1x1conv_1' % prefix)(x_1)
        x_1 = Activation('relu6')(x_1)

        x_2 = Conv2D(filters=branch_channels, kernel_size=1, strides=1, padding='same',
                     use_bias=False, name='%s/1x1conv_2' % prefix)(inputs)
        x_2 = BatchNormalization(axis=bn_axis, name='%s/bn_1x1conv_2' % prefix)(x_2)
        x_2 = Activation('relu6')(x_2)
        x_2 = DepthwiseConv2D(kernel_size=3, strides=2, padding='same',
                              use_bias=False, name='%s/3x3dwconv_2' % prefix)(x_2)
        x_2 = BatchNormalization(axis=bn_axis, name='%s/bn_3x3dwconv_2' % prefix)(x_2)
        x_2 = Conv2D(filters=branch_channels, kernel_size=1, strides=1, padding='same',
                     use_bias=False, name='%s/1x1conv_3' % prefix)(x_2)
        x_2 = BatchNormalization(axis=bn_axis, name='%s/bn_1x1conv_3' % prefix)(x_2)
        x_2 = Activation('relu6')(x_2)

        x = Concatenate(axis=bn_axis, name='%s/concat' % prefix)([x_1, x_2])

    if strides == 1:
        c_hat, c = channel_split(inputs, name='%s/split' % prefix)

        c = Conv2D(filters=branch_channels, kernel_size=1, strides=1, padding='same',
                   use_bias=False, name='%s/1x1conv_4' % prefix)(c)
        # c = BatchNormalization(axis=bn_axis, name='%s/bn_1x1conv_4' % prefix)(c)
        # c = Activation('relu6')(c)
        c = DepthwiseConv2D(kernel_size=3, strides=1, padding='same',
                            use_bias=False, name='%s/3x3dwconv_3' % prefix)(c)
        c = BatchNormalization(axis=bn_axis, name='%s/bn_3x3dwconv_3' % prefix)(c)
        # c = Activation('relu6')(c)
        c = return_activation(c, nl)
        # 引入注意力机制
        if sq:
            c = squeeze(c)
        # 下降通道数
        c = Conv2D(filters=branch_channels, kernel_size=1, strides=1, padding='same',
                   use_bias=False, name='%s/1x1conv_5' % prefix)(c)
        c = BatchNormalization(axis=bn_axis, name='%s/bn_1x1conv_4' % prefix)(c)
        x = Concatenate(axis=bn_axis, name='%s/concat' % prefix)([c_hat, c])

    x = Lambda(channel_shuffle, name='%s/channel_shuffle' % prefix)(x)

    return x


def exblock(inputs, out_channels, sq, stage=1, block=1):
    prefix = 'stage%d/block%d' % (stage, block)

    residual = Conv2D(out_channels, (1, 1), strides=(2, 2), padding='same', use_bias=False)(inputs)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(out_channels, (3, 3), padding='same', use_bias=False, name='%s/_sepconv1' % prefix)(inputs)
    x = BatchNormalization(name='%s/_sepconv1_bn' % prefix)(x)
    x = Activation('hard_swish', name='%s/_sepconv2_ac_hs' % prefix)(x)
    x = SeparableConv2D(out_channels, (3, 3), padding='same', use_bias=False, name='%s/_sepconv2' % prefix)(x)
    # 引入注意力机制
    if sq:
        x = squeeze(x)

    x = BatchNormalization(name='%s/_sepconv2_bn' % prefix)(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='%s/_pool' % prefix)(x)
    x = layers.add([x, residual])

    return x


def inception_unit(inputs, channel1, channel2, channel3, ):
    branch_0 = Conv2D(channel1, (1, 1), strides=(1, 1), padding='same', use_bias=False)(inputs)
    branch_0 = BatchNormalization(axis=-1, scale=False, name='stage1X1_1BN')(branch_0)
    branch_0 = Activation('relu6', name='stage1X1_1ac')(branch_0)

    branch_1 = Conv2D(channel2, (3, 3), strides=(1, 1), padding='same', use_bias=False)(inputs)
    branch_1 = BatchNormalization(axis=-1, scale=False, name='stage3X3_1BN')(branch_1)
    branch_1 = Activation('relu6', name='stage3X3_1ac')(branch_1)

    branch_pool = AveragePooling2D(3, strides=1, padding='same')(inputs)
    branch_pool = Conv2D(channel3, (1, 1), strides=(1, 1), padding='same', use_bias=False)(branch_pool)
    branch_pool = BatchNormalization(axis=-1, scale=False, name='stagep1X1_1BN')(branch_pool)
    branch_pool = Activation('relu6', name='stagep1X1_1ac')(branch_pool)

    branches = [branch_0, branch_1, branch_pool]

    x = Concatenate(name='mixed_5b')(branches)

    return x


def qzynetnew(input_shape=[256, 256, 3], classes=2,target=1):
    input_shape = [256, 256, 3]

    img_input = Input(shape=input_shape)

    x = Conv2D(32, (3, 3), strides=(1, 1), padding='same', use_bias=False)(img_input)
    x = BatchNormalization(axis=-1, scale=False, name='stage0.1X1_1BN')(x)
    x = Activation('relu6', name='stage0.1X1_1ac')(x)
    x = Conv2D(64, (3, 3), strides=(2, 2), padding='same', use_bias=False)(x)
    x = BatchNormalization(axis=-1, scale=False, name='stage00.1X1_1BN')(x)
    x = Activation('relu6', name='stage00.1X1_1ac')(x)
    x = MaxPooling2D(3, strides=1)(x)#2
    #   x=_shuffle_unit(x, 128, sq=False, nl='RE',strides=1, stage=2, block=1)
    #   x=_shuffle_unit(x, 128, sq=False, nl='RE',strides=1, stage=2, block=2)

    #   x=_shuffle_unit(x, 128, sq=False, nl='RE',strides=2, stage=2, block=3)#128,128,128 -> 64 x 64 x 128

    #   x=_shuffle_unit(x, 128, sq=False, nl='RE',strides=1, stage=2, block=4)
    #   x=_shuffle_unit(x, 128, sq=False, nl='RE',strides=1, stage=2, block=5)

    x = exblock(x, 128, sq=True, stage=1, block=1)
    f2=x

    x = exblock(x, 192, sq=True, stage=1, block=2)
    # x=_shuffle_unit(x, 256, sq=False, nl='RE',strides=2, stage=2, block=6)#64,64,128 -> 32 x 32 x 256
    x = inception_unit(x, 116, 116, 24)
    x = _shuffle_unit(x, 256, sq=False, nl='RE', strides=1, stage=2, block=7)
    x = _shuffle_unit(x, 256, sq=False, nl='RE', strides=1, stage=2, block=8)
    f3= x

    x = _shuffle_unit(x, 512, sq=False, nl='RE', strides=2, stage=2, block=9)  # 32,32,256 -> 16 x 16 x 512

    x = _shuffle_unit(x, 512, sq=False, nl='RE', strides=1, stage=2, block=10)
    x = _shuffle_unit(x, 512, sq=False, nl='RE', strides=1, stage=2, block=11)
    x = _shuffle_unit(x, 512, sq=False, nl='RE', strides=1, stage=2, block=12)
    f4= x

    x = _shuffle_unit(x, 1024, sq=True, nl='HS', strides=2, stage=2, block=13)  # 16 x 16 x 512 -> 8 x 8 x 1024

    x = _shuffle_unit(x, 1024, sq=True, nl='HS', strides=1, stage=2, block=14)
    f5= x

    if target == 1:
         x = GlobalAveragePooling2D(name='global_max_pool')(x)
         x = Dense(classes, name='fc')(x)
         x = Activation('sigmoid')(x)
         inputs = img_input
         model = Model(inputs, x, name='qzynet')
         return model

    if target == 2:
         return img_input, [f2, f3, f4, f5]

In [ ]:
model_final = qzynetnew()

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
#         val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_predict = np.argmax(np.asarray(self.model.predict(self.validation_data[0])), axis=1)
#         val_targ = self.validation_data[1]
        val_targ = np.argmax(self.validation_data[1], axis=1)
        _val_f1 = f1_score(val_targ, val_predict, average='macro')
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print('— val_f1: %f — val_precision: %f — val_recall %f' %(_val_f1, _val_precision, _val_recall))
#         print(' — val_f1:' ,_val_f1)
        return

metrics1 = Metrics()

In [ ]:
# model_final.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=[f1,recall1,'acc'])
model_final.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('boat_detector')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', epsilon=0.01, cooldown=0, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=2) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat,metrics1]

In [ ]:
from keras import optimizers
def fit():
    epochs = 40
    lrate = 0.01
    decay = lrate/epochs
    #adam = optimizers.Adam(lr=lrate,beta_1=0.9, beta_2=0.999, decay=decay)
    sgd = optimizers.SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model_final.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['binary_accuracy'])
    loss_history=[model_final.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=40, batch_size=50,callbacks=callbacks_list)]
    
    return loss_history
num=0

while True:
    num=num+1
#     prefix='%d'%(num)
    loss_history = fit()
    model_final.save_weights('my_model_weights%d.h5'% num)
    if np.min([mh.history['val_loss'] for mh in loss_history]) < 0.1:
        break
    if num==1:
        break

In [ ]:
def show_loss(loss_history):
    epochs = np.concatenate([mh.epoch for mh in loss_history])
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 10))
    
    _ = ax1.plot(epochs, np.concatenate([mh.history['loss'] for mh in loss_history]), 'b-',
                 epochs, np.concatenate([mh.history['val_loss'] for mh in loss_history]), 'r-')
    ax1.legend(['Training', 'Validation'])#图表，损失函数（训练和验证）的迭代图表
    ax1.set_title('Loss')
    
    _ = ax2.plot(epochs, np.concatenate([mh.history['binary_accuracy'] for mh in loss_history]), 'b-',
                 epochs, np.concatenate([mh.history['val_binary_accuracy'] for mh in loss_history]), 'r-')
    ax2.legend(['Training', 'Validation'])#准确率，（训练和迭代的）
    ax2.set_title('Binary Accuracy (%)')

show_loss(loss_history)

In [ ]:
# model_final.load_weights(weight_path)#读取权重
# model_final.save_weights('my_model_weights.h5')

准备预测数据

In [ ]:
unique_img_ids.shape[0]

In [ ]:
unique_img_ids1 = unique_img_ids[20000:30000]

In [ ]:
unique_img_ids1.shape[0]

In [ ]:

x_test = np.empty(shape=(10000, 256,256,3),dtype=np.uint8)#10680 256
y_test = np.empty(shape=10000,dtype=np.uint8)
for index, image in enumerate(unique_img_ids1['ImageId']):
    image_array= Image.open('../input/airbus-ship-detection/train_v2/' + image).resize((256,256)).convert('RGB') #256
    x_test[index] = image_array
    y_test[index]=unique_img_ids1[unique_img_ids1['ImageId']==image]['has_ship'].iloc[0]

print(x_test.shape)
print(y_test.shape)

In [ ]:
y_test_targets =y_test.reshape(len(y_test),-1)
enc = OneHotEncoder()
enc.fit(y_test_targets)
y_test = enc.transform(y_test_targets).toarray()
print(y_test.shape)

In [ ]:
predict_ship = model_final.evaluate( x_test,y_test)

In [ ]:
acc=predict_ship[1]*100

In [ ]:
print ('Accuracy of random data = '+ str(acc) + "%")